# IMAGO TEST


## Task 1

#### **Data Analysis & Understanding**
    - You’ll receive three CSV files:
        - `invoices.csv` : represents `[Abrechnung_Rechnungen]`
        - `positions.csv` : represents `[Abrechnung_Positionen]`
        - `customers.csv` : customer lookup with `KdNr`, `VerlagsName`, and `Region`
    - You're free to create mock data based on the schema.
    - Using SQL or Python (e.g. pandas), analyze:
        - How many positions are linked to invoices that are missing payment info
        - How much revenue is attributed to placeholder media ID '100000000'
        - How many invoices have no positions attached
    - Document your findings in a short markdown file or Jupyter Notebook.

###### Importing data from csv

In [2]:
import pandas as pd

I will use read_csv to import data from csv files into a dataframe pandas, i will use sep=';' because we can see from file that the separator character is ';' 

In [4]:
Abrechung_Rechnungen = pd.read_csv(r'C:\Users\itali\Downloads\data-coding-challenge\data-coding-challenge\invoices.csv',sep=';')
Abrechung_Positionen = pd.read_csv(r'C:\Users\itali\Downloads\data-coding-challenge\data-coding-challenge\positions.csv',sep=';')
Abrechnung_Kunden = pd.read_csv(r'C:\Users\itali\Downloads\data-coding-challenge\data-coding-challenge\customers.csv',sep=';')

just have a look from the tables

In [8]:
Abrechung_Rechnungen.head()

,ReNummer,SummeNetto,MwStSatz,ZahlungsbetragBrutto,KdNr,Summenebenkosten,ReDatum,Zahlungsdatum
0,103601,11786.7,0,0.0,78911,0.0,2025-04-04 00:00:00.000,NaN
1,103600,379.0,7,0.0,79666,0.0,2025-04-03 00:00:00.000,NaN
2,103599,160.0,7,0.0,30145,0.0,2025-04-04 00:00:00.000,NaN
3,103598,225.0,7,0.0,10383,0.0,2025-04-01 00:00:00.000,NaN
4,103597,1260.0,7,0.0,20213,0.0,2025-04-03 00:00:00.000,NaN


In [9]:
Abrechung_Positionen.head()

,id,ReId,KdNr,Nettobetrag,Bildnummer,VerDatum
0,4154495,102038,50292.0,19.0,168161043.0,2022-09-29 00:00:00.000
1,4154496,102038,50292.0,19.0,108514408.0,2022-09-12 00:00:00.000
2,4154497,102038,50292.0,19.0,55374589.0,2022-09-08 00:00:00.000
3,4154498,102038,50292.0,19.0,55374589.0,2022-09-08 00:00:00.000
4,4154499,102038,50292.0,19.0,57865918.0,2022-09-08 00:00:00.000


In [10]:
Abrechnung_Kunden.head()

,id,Kdnr,Verlagsname,Region
0,5,20172,1. FC Nürnberg,Nürnberg
1,19,20137,Allgäuer Zeitung / Allgäuer Zeitungsverlag GmbH,Bodensee
2,27,20115,Augsburger Allgemeine,München
3,69,10113,Brainpool TV Productions,Rheinland
4,72,10154,RFW / Redaktionsbüro Wipperfürth,Hamburg


**FIRST QUESTION**

The first question is: **How many positions are linked to invoices that are missing payment info**

To answer this, I would like to identify, for each position, the invoices that have missing values in any of the following fields: Zahlungsdatum (Payment Date), SummeNetto (Net Amount), ZahlungsbetragBrutto (Gross Payment Amount), KdNr (Customer Number), and Summenebenkosten (Additional Costs). These fields, according to the schema, are allowed to be NULL. As a first step, I want to verify whether there are any missing values in these fields by using the isnull() function in combination with sum(). 

In [71]:
print('Zahlungsdatum:')
print(Abrechung_Rechnungen['Zahlungsdatum'].isnull().sum())
print('ZahlungsbetragBrutto:')
print(Abrechung_Rechnungen['ZahlungsbetragBrutto'].isnull().sum())
print('KdNr:')
print(Abrechung_Rechnungen['KdNr'].isnull().sum())
print('Summenebenkosten:')
print(Abrechung_Rechnungen['Summenebenkosten'].isnull().sum())
print('SummeNetto:')
print(Abrechung_Rechnungen['SummeNetto'].isnull().sum())

Zahlungsdatum:
399
ZahlungsbetragBrutto:
1
KdNr:
0
Summenebenkosten:
2
SummeNetto:
0


In [76]:
Abrechung_Rechnungen.loc[Abrechung_Rechnungen['ZahlungsbetragBrutto'].isnull()].head()

,ReNummer,SummeNetto,MwStSatz,ZahlungsbetragBrutto,KdNr,Summenebenkosten,ReDatum,Zahlungsdatum
1270,102331,1297.5,7,NaN,10957,0.0,2025-04-01 00:00:00.000,NaN


In [77]:
Abrechung_Rechnungen.loc[Abrechung_Rechnungen['Summenebenkosten'].isnull()].head()

,ReNummer,SummeNetto,MwStSatz,ZahlungsbetragBrutto,KdNr,Summenebenkosten,ReDatum,Zahlungsdatum
19,103582,350.0,7,0.0,10383,NaN,2025-04-03 00:00:00.000,NaN
21,103580,11000.0,7,0.0,20608,NaN,2025-04-03 00:00:00.000,NaN


Upon reviewing the data, I noticed that the two missing records both have Zahlungsdatum (Payment Date) as NULL. Given this, it may be sufficient to filter only on Zahlungsdatum IS NULL, unless there is a known correlation between missing payment dates and other fields such as SummeNetto, ZahlungsbetragBrutto, KdNr, or Summenebenkosten.

 Another scenario to consider is that some positions may not be linked to any invoices at all. However, since the question specifically asks about positions that are linked to invoices, I will use an inner join between the positions and invoices tables to exclude unlinked records.

There is a common field between the positions and invoices tables: KdNr, which likely stands for Kundennummer (Customer Number), as it also appears in the clients table. Additionally, ReNummer is the primary key of the invoices table, meaning that each ReNummer–KdNr pair must be unique. However, the positions table contains multiple rows with the same ReId but different KdNr values.
This inconsistency may indicate data quality issues and could explain problems in linking positions to the correct customers

So the next step will be, comparing the solution of two different queries to better understand the problem with KdNr:
- the first one is made up an inner join between positions and invoices using only the condition ReNummer = ReId
- the second one is made up an inner join between positions and invoices using ReNummer = ReId and positions.KdNr = invoices.KdNr

Before we want to check if there are also some missing values for KdNr in the position table  

In [155]:
Abrechung_Positionen.loc[Abrechung_Positionen['KdNr'].isnull()]['id'].nunique()

1

In [156]:
Abrechung_Positionen.loc[Abrechung_Positionen['KdNr'].isnull()].head()

,id,ReId,KdNr,Nettobetrag,Bildnummer,VerDatum
124096,5715403,103381,NaN,NaN,NaN,NaN


there is also one row in position table with a missing value for KdNr

**Solution n.1**

In [129]:
Positionen_inner_joined_Rechnungen = pd.merge(Abrechung_Positionen, Abrechung_Rechnungen, how='inner', left_on='ReId',right_on= 'ReNummer')

In [130]:
Positionen_inner_joined_Rechnungen.head()

,id,ReId,KdNr_x,Nettobetrag,Bildnummer,VerDatum,ReNummer,SummeNetto,MwStSatz,ZahlungsbetragBrutto,KdNr_y,Summenebenkosten,ReDatum,Zahlungsdatum
0,4154495,102038,50292.0,19.0,168161043.0,2022-09-29 00:00:00.000,102038,7114.46,0,6852.62,79705,261.84,2024-12-09 00:00:00.000,2025-01-14 00:00:00.000
1,4154496,102038,50292.0,19.0,108514408.0,2022-09-12 00:00:00.000,102038,7114.46,0,6852.62,79705,261.84,2024-12-09 00:00:00.000,2025-01-14 00:00:00.000
2,4154497,102038,50292.0,19.0,55374589.0,2022-09-08 00:00:00.000,102038,7114.46,0,6852.62,79705,261.84,2024-12-09 00:00:00.000,2025-01-14 00:00:00.000
3,4154498,102038,50292.0,19.0,55374589.0,2022-09-08 00:00:00.000,102038,7114.46,0,6852.62,79705,261.84,2024-12-09 00:00:00.000,2025-01-14 00:00:00.000
4,4154499,102038,50292.0,19.0,57865918.0,2022-09-08 00:00:00.000,102038,7114.46,0,6852.62,79705,261.84,2024-12-09 00:00:00.000,2025-01-14 00:00:00.000


In [131]:
len(Positionen_inner_joined_Rechnungen)

129092

Now we can filter for null values and count the number of positions. We want to perform a select count(distinct id) filtered by the condition Zahlungsdatum is null. So we will use the loc combined to isnull to filter and the nunique to count the number of unique values in the column id (position id)

In [132]:
Positionen_inner_joined_Rechnungen.loc[
    Positionen_inner_joined_Rechnungen['Zahlungsdatum'].isnull() 
].id.nunique()

18011

another way could be to calculate the len of the array of unique position id values

In [133]:
len(Positionen_inner_joined_Rechnungen.loc[Positionen_inner_joined_Rechnungen['Zahlungsdatum'].isnull()]['id'].unique())

18011

If we want to apply this type of analysis more generally, we can modify the Python script as shown below, including an or condition between all the fields that could potentially be null.

In [135]:
Positionen_inner_joined_Rechnungen.loc[
    Positionen_inner_joined_Rechnungen['Zahlungsdatum'].isnull() |
    Positionen_inner_joined_Rechnungen['ZahlungsbetragBrutto'].isnull() |
    Positionen_inner_joined_Rechnungen['KdNr_y'].isnull() |
    Positionen_inner_joined_Rechnungen['Summenebenkosten'].isnull() |
    Positionen_inner_joined_Rechnungen['SummeNetto'].isnull()
].id.nunique()

18011

**Solution n.2**

In [115]:
Positionen_inner_joined_Rechnungen = pd.merge(Abrechung_Positionen, Abrechung_Rechnungen, how='inner', left_on=['ReId','KdNr'],right_on= ['ReNummer','KdNr'])

In [116]:
Positionen_inner_joined_Rechnungen.head()

,id,ReId,KdNr,Nettobetrag,Bildnummer,VerDatum,ReNummer,SummeNetto,MwStSatz,ZahlungsbetragBrutto,Summenebenkosten,ReDatum,Zahlungsdatum
0,5387224,101604,20843.0,4.5,1.039632e+09,2024-10-01 00:00:00.000,101604,94.5,7,101.12,0.0,2024-11-04 00:00:00.000,2024-11-18 00:00:00.000
1,5387225,101604,20843.0,4.5,1.044140e+09,2024-10-01 00:00:00.000,101604,94.5,7,101.12,0.0,2024-11-04 00:00:00.000,2024-11-18 00:00:00.000
2,5387226,101604,20843.0,4.5,1.014992e+09,2024-10-01 00:00:00.000,101604,94.5,7,101.12,0.0,2024-11-04 00:00:00.000,2024-11-18 00:00:00.000
3,5387227,101604,20843.0,4.5,1.011300e+09,2024-10-01 00:00:00.000,101604,94.5,7,101.12,0.0,2024-11-04 00:00:00.000,2024-11-18 00:00:00.000
4,5387228,101604,20843.0,4.5,3.832862e+07,2024-10-01 00:00:00.000,101604,94.5,7,101.12,0.0,2024-11-04 00:00:00.000,2024-11-18 00:00:00.000


In [117]:
len(Positionen_inner_joined_Rechnungen)

58206

We can see here that the rows that have a match with also KdNr are less than the others

In [118]:
Positionen_inner_joined_Rechnungen.loc[
    Positionen_inner_joined_Rechnungen['Zahlungsdatum'].isnull() 
].id.nunique()

7112

If we want to apply this type of analysis more generally, we can modify the Python script as shown below, including an or condition between all the fields that could potentially be null.

In [120]:
Positionen_inner_joined_Rechnungen.loc[
    Positionen_inner_joined_Rechnungen['Zahlungsdatum'].isnull() |
    Positionen_inner_joined_Rechnungen['ZahlungsbetragBrutto'].isnull() |
    Positionen_inner_joined_Rechnungen['KdNr'].isnull() |
    Positionen_inner_joined_Rechnungen['Summenebenkosten'].isnull() |
    Positionen_inner_joined_Rechnungen['SummeNetto'].isnull()
].id.nunique()

7112

After analyzing the data structure and constraints, I identified a potential issue with the KdNr field. Since ReNummer is the primary key in the invoices table, each invoice (ReNummer) must be unique and should not be associated with multiple customer numbers (KdNr).

However, in the positions table, some rows reference the same ReId with different KdNr values, which may indicate a data integrity issue.

Despite this, based on the join logic and the requirement to find positions linked to invoices with missing payment information, the correct number of such positions is **18,011**. This result is returned by **Solution 1**, which uses the join condition only on ReNummer(ReId)

**SECOND QUESTION**

The second question is: **How much revenue is attributed to placeholder media ID '100000000'**

The 'media ID' could be translated with the column 'Bildnummer' in position table and the revenue is linked to Nettobetrag.

I want to perform an inner join between positions and invoices to link media id to the revenue, after i will filter for media id = 100000000 (where media_id=100000000) and sum the revenue (sum(revenue)). In this case we will use an inner join only on ReNummer ('ReId' for positions)

So we can use again the previous inner join between positions and invoices filter the data for the bildnummer 100000000 and after sum the Nettobetrag  

In [139]:
Positionen_inner_joined_Rechnungen = pd.merge(Abrechung_Positionen, Abrechung_Rechnungen, how='inner', left_on='ReId',right_on= 'ReNummer')

In [140]:
#100000000
Positionen_inner_joined_Rechnungen.loc[Positionen_inner_joined_Rechnungen['Bildnummer'] == 100000000]['Nettobetrag'].sum()

1319897.9100000001

So the revenue attributed to placeholder media ID '100000000' is: **1319897.9100000001**

if we will join also with KdNr field, we will have a lower amount

In [141]:
Positionen_inner_joined_Rechnungen = pd.merge(Abrechung_Positionen, Abrechung_Rechnungen, how='inner', left_on=['ReId','KdNr'],right_on= ['ReNummer','KdNr'])

In [142]:
Positionen_inner_joined_Rechnungen.loc[Positionen_inner_joined_Rechnungen['Bildnummer'] == 100000000]['Nettobetrag'].sum()

720639.97

**THIRD QUESTION**

The third question is: **How many invoices have no positions attached**

I want to perform a left join between invoices and positions and find the invoices that have no positions attached by finding the rows that have position id null

In [143]:
Rechnungen_left_joined_Positionen = pd.merge(Abrechung_Rechnungen, Abrechung_Positionen, how='left', left_on = ['ReNummer'], right_on = ['ReId'])

In [144]:
Rechnungen_left_joined_Positionen.head()

,ReNummer,SummeNetto,MwStSatz,ZahlungsbetragBrutto,KdNr_x,Summenebenkosten,ReDatum,Zahlungsdatum,id,ReId,KdNr_y,Nettobetrag,Bildnummer,VerDatum
0,103601,11786.7,0,0.0,78911,0.0,2025-04-04 00:00:00.000,NaN,4516297.0,103601.0,78153.0,741.64,100000000.0,2023-01-15 00:00:00.000
1,103601,11786.7,0,0.0,78911,0.0,2025-04-04 00:00:00.000,NaN,4516298.0,103601.0,78152.0,14.68,113020829.0,2023-01-15 00:00:00.000
2,103601,11786.7,0,0.0,78911,0.0,2025-04-04 00:00:00.000,NaN,4516299.0,103601.0,78152.0,14.68,87092278.0,2023-01-15 00:00:00.000
3,103601,11786.7,0,0.0,78911,0.0,2025-04-04 00:00:00.000,NaN,4516300.0,103601.0,78152.0,14.68,130661743.0,2023-01-15 00:00:00.000
4,103601,11786.7,0,0.0,78911,0.0,2025-04-04 00:00:00.000,NaN,4516301.0,103601.0,78152.0,14.68,87092278.0,2023-01-15 00:00:00.000


In [145]:
Rechnungen_left_joined_Positionen.loc[Rechnungen_left_joined_Positionen['id'].isnull()].ReNummer.nunique()

2

So the invoices that have no positions attached ar 2

In this case if we want to join also with KdNr we will have more rows without position attached

In [147]:
Rechnungen_left_joined_Positionen = pd.merge(Abrechung_Rechnungen, Abrechung_Positionen, how='left', left_on = ['ReNummer','KdNr'], right_on = ['ReId','KdNr'])

In [149]:
Rechnungen_left_joined_Positionen.head()

,ReNummer,SummeNetto,MwStSatz,ZahlungsbetragBrutto,KdNr,Summenebenkosten,ReDatum,Zahlungsdatum,id,ReId,Nettobetrag,Bildnummer,VerDatum
0,103601,11786.7,0,0.0,78911,0.0,2025-04-04 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN
1,103600,379.0,7,0.0,79666,0.0,2025-04-03 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN
2,103599,160.0,7,0.0,30145,0.0,2025-04-04 00:00:00.000,NaN,5726648.0,103599.0,160.0,1.038478e+09,2025-04-04 00:00:00.000
3,103598,225.0,7,0.0,10383,0.0,2025-04-01 00:00:00.000,NaN,5726249.0,103598.0,45.0,1.344517e+08,2025-03-10 00:00:00.000
4,103598,225.0,7,0.0,10383,0.0,2025-04-01 00:00:00.000,NaN,5726250.0,103598.0,45.0,1.059388e+08,2025-03-15 00:00:00.000


In [150]:
Rechnungen_left_joined_Positionen.loc[Rechnungen_left_joined_Positionen['id'].isnull()].ReNummer.nunique()

1011